<h1>Capstone Project - Segmenting Hospitals in Greater London based on distance</h1>

<h3>Import all the necessary libraries</h3>

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

<h1>List all the hospitals in Greater London, UK</h1>

<h3>Connect FourSquare API to extract the details</h3>

In [84]:
CLIENT_ID = 'ZVPHYUXAOWDL1WASNXQZ3DOEHFZ4DTO2HFO0REQSPZXU0PDV'
CLIENT_SECRET = 'MEVWT2NFOQN3RPHM1IX0G3CGUQUQCTNQ3FCCYNJWRZ4AHM2Q'
VERSION = '20180605'
LIMIT=5000
radius=100000
address = 'Bangalore'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
query = 'hospital'
fsurl = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(CLIENT_ID,
                                                                                                                           CLIENT_SECRET,
                                                                                                                           VERSION,
                                                                                                                          latitude,
                                                                                                                          longitude,query,LIMIT,radius)
results = requests.get(fsurl).json()
results

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


{'meta': {'code': 200, 'requestId': '5bcd90f8f594df1e004bc2db'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bangalore',
  'headerFullLocation': 'Bangalore',
  'headerLocationGranularity': 'city',
  'query': 'hospital',
  'totalResults': 39,
  'suggestedBounds': {'ne': {'lat': 13.024119845000044,
    'lng': 77.63739332570279},
   'sw': {'lat': 12.934119754999955, 'lng': 77.5452060742972}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d2c2717888af04d8ad0e5af',
       'name': 'Vikram Hospital',
       'location': {'address': '71/1 Millers Rd',
        'lat': 12.988449594584456,
        'lng': 77.59199314905337,
        'labeledLatLngs': [{'label': 'display',
          'lat': 

<h3>List down all the hospitals with distance from Greater London</h3>

In [103]:
venues = results['response']['groups'][0]['items']
hospitals = json_normalize(venues)
columns = ['venue.name','venue.location.address','venue.location.city','venue.location.distance', 'venue.location.lat', 'venue.location.lng']
hospitals =hospitals.loc[:, columns]
hospitals.columns = [col.split(".")[-1] for col in hospitals.columns]
hospitals.city.fillna('Bangalore', inplace=True)
hospitals

,name,address,city,distance,lat,lng
0,Vikram Hospital,71/1 Millers Rd,Bangalore,1041,12.988450,77.591993
1,Fortis Hospitals,"Cunningham Rd, Vasanth Nagar",Bangalore,1045,12.988082,77.594183
2,Sparsh Hospital,NaN,Bangalore,599,12.983105,77.595015
3,St. Martha's Hospital,NaN,Bangalore,1266,12.969271,77.585446
4,Shri Bhagwan Mahaveer Jain Hospital,Vasanth Nagar,Bangalore,1522,12.992345,77.594896
5,Mallya Hospital,"2,Vittal Mallya road",Bangalore,1288,12.968082,77.594863
6,Mallige Hospital & Heart Centre,Race Course Crescent Road,Bangalore,1378,12.985195,77.580224
7,HCG Oncology,"HCG Towers, #8, P. Kalinga Rao Road,",Bangalore,1662,12.964278,77.589562
8,Republic Hospital,5 Langford Gardens,Bangalore,1792,12.964401,77.597997
9,Bowring & Lady Curzon Hospital,Tasker Town,Bangalore,1510,12.982437,77.604804


<h3>K-means Algorithm and clusters map</h3>

In [ ]:
#Run k-means 

# set number of clusters
kclusters = 4

grouped_clustering = hospitals.drop(['name','address','city','lat','lng'],axis=1,inplace=False)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# create a new dataframe that includes the cluster as well .
blre_merged = hospitals

# add clustering labels
blre_merged['Cluster Labels'] = kmeans.labels_
blre_merged

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
rainbow = ['#8000ff', '#2adddd', '#000000', '#ff0000']

# add markers to the map
markers_colors = []
for lat, lng, name, postalcode,cluster in zip(hospitals['lat'], hospitals['lng'], hospitals['name'],hospitals['city'], hospitals['Cluster Labels']):
    label = folium.Popup(str(name) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

<h3>Cluster Details</h3>

In [102]:
blre_merged.loc[blre_merged['Cluster Labels'] == 0, blre_merged.columns[[0] + list(range(2, blre_merged.shape[1]))]]

,name,city,distance,lat,lng,Cluster Labels
21,Rangadore Memorial Hospital,Bangalore,3585,12.953554,77.571196,0
22,Agadi Hospital,Bangalore,3414,12.949084,77.597690,0
23,Shankara Cancer Hospital,Bangalore,3523,12.954407,77.571001,0
24,"St. Martha's Heart Centre, Nrupathunga Road",Bangalore,3580,12.988336,77.559672,0
25,Abhaya Hospital,Bangalore,3628,12.946674,77.594467,0
26,Maiya Hospital,NaN,4116,12.942596,77.585379,0


In [101]:
blre_merged.loc[blre_merged['Cluster Labels'] == 1, blre_merged.columns[[0] + list(range(2, blre_merged.shape[1]))]]

,name,city,distance,lat,lng,Cluster Labels
0,Vikram Hospital,Bangalore,1041,12.988450,77.591993,1
1,Fortis Hospitals,Bangalore,1045,12.988082,77.594183,1
2,Sparsh Hospital,NaN,599,12.983105,77.595015,1
3,St. Martha's Hospital,NaN,1266,12.969271,77.585446,1
4,Shri Bhagwan Mahaveer Jain Hospital,Bangalore,1522,12.992345,77.594896,1
5,Mallya Hospital,Bangalore,1288,12.968082,77.594863,1
6,Mallige Hospital & Heart Centre,Bangalore,1378,12.985195,77.580224,1
7,HCG Oncology,Bangalore,1662,12.964278,77.589562,1
8,Republic Hospital,Bangalore,1792,12.964401,77.597997,1
9,Bowring & Lady Curzon Hospital,Bangalore,1510,12.982437,77.604804,1


In [100]:
blre_merged.loc[blre_merged['Cluster Labels'] == 2, blre_merged.columns[[0] + list(range(2, blre_merged.shape[1]))]]

,name,city,distance,lat,lng,Cluster Labels
27,Cloudnine,NaN,4458,13.007732,77.562538,2
28,NIMHANS,Bangalore,4470,12.939091,77.594651,2
29,Command Hospital Air Force,Bangalore,4402,12.962985,77.628353,2
30,Kidwai Memorial Institute Of Oncology,NaN,4632,12.938066,77.598266,2
31,Anugraha Vittala Hospital,NaN,4709,12.938721,77.578417,2
32,Apollo Hospital,NaN,4788,12.936649,77.584307,2
33,Bangalore Hospital,Bangalore,4792,12.937468,77.580128,2
34,Ramakrishna Hospital,Bangalore,4817,12.936542,77.583376,2
35,ESI Crossing,NaN,4909,12.971615,77.635901,2
36,Vasan Eye Care,Bangalore,4915,12.999373,77.551035,2


In [99]:
blre_merged.loc[blre_merged['Cluster Labels'] == 3, blre_merged.columns[[0] + list(range(2, blre_merged.shape[1]))]]

,name,city,distance,lat,lng,Cluster Labels
11,St. Philomenas Hospital,Bangalore,2571,12.965635,77.610545,3
13,Narayana Nethralaya,Bangalore,2369,12.966955,77.609227,3
14,Santosh Hospital,Bangalore,2626,12.996150,77.608054,3
15,Hosmat Hospital,Bangalore,2654,12.968852,77.613390,3
16,Seventh Day Adventist Hospital,Bangalore,2839,12.995457,77.611398,3
17,leprosy hospital,NaN,2901,12.975363,77.564833,3
18,Lakeside Hospital,Bangalore,3033,12.988163,77.617685,3
19,KC General Hospital,NaN,3078,12.996801,77.569479,3
20,K.I.M.S Hospital & Research Center,NaN,3135,12.956338,77.574309,3
